In [1]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time


In [22]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
import time
import json


# ChromeDriver 경로 설정
service = Service('/usr/local/bin/chromedriver')

# Chrome 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')  # WSL 환경에서 필수
#options.add_argument('--disable-dev-shm-usage')  # 공유 메모리 비활성화
#options.add_argument('--headless')  # GUI가 필요 없는 경우W

# WebDriver 실행 service=service,
driver = webdriver.Chrome( options=options)

# 잡코리아 데이터사이언티스트+머신러닝엔지니어  홈페이지 열기
driver.get('https://www.jobkorea.co.kr/recruit/joblist?menucode=duty')
print("Page title:", driver.title)
time.sleep(1)

# 첫 번째 단계: groupName이 '개발·데이터'인 요소를 클릭
group_name_element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//li[@class='item' and contains(@data-value-json, '\"groupName\":\"개발·데이터\"')]"))
)
group_name_element.click()

# 대기: 콘텐츠가 동적으로 로드되기를 기다림
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//li[@class='item' and contains(@data-value-json, '\"subName\":\"데이터사이언티스트\"')]"))
)

# XPath를 사용하여 체크박스를 찾고 클릭
# XPath로 input 태그 찾기

# data-name="데이터사이언티스트"
checkbox_datascientist = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//input[@type='checkbox' and @data-name='데이터사이언티스트']"))
)
driver.execute_script("arguments[0].click();", checkbox_datascientist)

time.sleep(0.5)
# data-name="머신러닝엔지니어"
checkbox_mlengineer = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH, "//input[@type='checkbox' and @data-name='머신러닝엔지니어']"))
)
driver.execute_script("arguments[0].click();", checkbox_mlengineer)


time.sleep(0.5)

# 기존 innerHTML 상태 저장
previous_state = driver.find_element(By.ID, "dev-gi-list").get_attribute("innerHTML")

# 'dev-btn-search' 버튼을 클릭
search_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, "dev-btn-search"))
)
search_button.click()
time.sleep(0.5)

# # innerHTML의 변화를 기다림
WebDriverWait(driver, 10).until(
    lambda d: d.find_element(By.ID, "dev-gi-list").get_attribute("innerHTML") != previous_state
)
# id가 'dev-gi-list'인 div 태그 내부에서 href 값 추출
href_list = []

# 페이지 넘김 로직
pagination_id = "dvGIPaging"

# 최대 페이지 개수를 설정 (필요에 따라 동적으로 계산)
max_pages = 12  # 예시로 5페이지만 처리

for j in range(max_pages):
    print('='*30)
    print(f'현재 페이지: {j+1}')
    if j+1 > 1:
        try:
            if (j + 1) % 10 == 0:
                # 10의 배수일 경우 다음 그룹으로 이동
                next_group_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[@id='{pagination_id}']//a[contains(@class, 'tplBtn btnPgnNext')]"))
                )
                next_group_button.click()
                print(f"다음 그룹으로 이동 (페이지 {j+2})")
            else:
                # data-page의 값이 j+2인 요소를 클릭
                next_page_element = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[@id='{pagination_id}']//a[@data-page='{j+2}']"))
                )
                next_page_element.click()
                time.sleep(2)  # 페이지가 로드되도록 대기
        except TimeoutException:
            print(f"페이지 {j+2}를 찾을 수 없습니다.")
            break

    # 현재 페이지에서 href 값을 수집
    a_elements = driver.find_elements(By.XPATH, "//div[@id='dev-gi-list']//tr//td[@class='tplTit']//a[@href]")
    for i, a_element in enumerate(a_elements):
        href = a_element.get_attribute("href")
        href_list.append(href)
        print(f"추출된 href ({i+1}/{len(a_elements)}): {href}")

print(f"총 {len(href_list)}개의 href가 추출되었습니다.")

all_text_data = []
all_table_data = []
all_tag_data = []
all_data = []
for i, url in enumerate(href_list):
        # iframe에 접근
    
    print('='*20)
    print(f"접속 중: {url} ({i+1}/{len(href_list)})")
    driver.get(url)
    print(f"페이지 제목: {driver.title}")

      # 최종 모든 데이터를 저장할 리스트


    article_text = "N/A"  # 기본값 설정
    table_data = "N/A"  # 기본값 설정

    # id가 'gib_frame'인 iframe을 찾아 전환
    try:
        iframe = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "gib_frame"))  # 특정 id를 가진 iframe만 찾기
        )
        driver.switch_to.frame(iframe)  # iframe으로 전환
        print("id='gib_frame'인 iframe으로 전환 완료.")
    except TimeoutException:
        print("id='gib_frame'인 iframe이 존재하지 않습니다. 메인 페이지에서 데이터 추출 시도.")

    # article 데이터 추출
    try:
        article_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//article[contains(@class, 'view-content') and contains(@class, 'view-detail') and contains(@class, 'dev-wrap-detailContents')]"))
        )
        article_text = article_element.text.strip()
        print("article 태그 내용 추출 성공")

        # article 데이터를 딕셔너리에 저장
        all_article_data = {
            "title": driver.title,
            "url": url,
            "article_text": article_text
        }
        all_data.append(all_article_data)  # all_data 리스트에 추가
    except TimeoutException:
        print("해당 클래스명을 가진 article 태그를 찾을 수 없습니다.")

    # 테이블 데이터 추출
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//table"))
        )
        tables = driver.find_elements(By.TAG_NAME, "table")  # 모든 table 태그 찾기
        extracted_table_data = []

        for table_index, table in enumerate(tables):
            rows = table.find_elements(By.XPATH, ".//tr")  # 각 table의 모든 tr 태그 찾기
            table_data = []

            for row_index, row in enumerate(rows):
                row_data = []
                cells = row.find_elements(By.XPATH, ".//td | .//th")  # td와 th 태그 찾기
                for cell in cells:
                    try:
                        # 셀 내부 모든 텍스트를 가져오기 (중첩된 태그 포함)
                        cell_text = cell.get_attribute("innerText").strip()
                        row_data.append(cell_text if cell_text else "N/A")
                    except StaleElementReferenceException:
                        print("StaleElementReferenceException 발생. 셀을 건너뜁니다.")
                        row_data.append("N/A")
                table_data.append(row_data)

            extracted_table_data.append({
                "table_index": table_index,
                "table_data": table_data
            })

        # table 데이터를 딕셔너리에 저장
        all_table_data = {
            "title": driver.title,
            "url": url,
            "tables": extracted_table_data
        }
        all_data.append(all_table_data)  # all_data 리스트에 추가

    except TimeoutException:
        print("페이지에 테이블이 없습니다.")

    # iframe에서 메인 페이지로 돌아오기
    driver.switch_to.default_content()
    time.sleep(1)  # 페이지가 완전히 로드되도록 잠시 대기

    time.sleep(0.5)
    if i==4:
        break

# 최종 데이터 출력
print('전체 데이터:', json.dumps(all_data, ensure_ascii=False, indent=4))

# JSON 파일로 저장
output_file_name = f'all_data_{time.localtime().tm_mon}{time.localtime().tm_mday}{time.localtime().tm_hour}{time.localtime().tm_min}{time.localtime().tm_sec}.json'
with open(output_file_name, "w", encoding="utf-8") as f:
    json.dump(all_data, f, ensure_ascii=False, indent=4)

print(f"JSON 파일 저장 완료: {output_file_name}")
     
#      # id가 'gib_frame'인 iframe을 찾아 전환
#     try:
#         iframe = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.ID, "gib_frame"))  # 특정 id를 가진 iframe만 찾기
#         )
#         driver.switch_to.frame(iframe)  # iframe으로 전환
#         print("id='gib_frame'인 iframe으로 전환 완료.")
#     except TimeoutException:
#         print("id='gib_frame'인 iframe이 존재하지 않습니다. 메인 페이지에서 데이터 추출 시도.")

#      # 페이지 로드 대기
#     WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.TAG_NAME, "body"))
#     )
   
    
#     # 전체 텍스트 데이터 수집
#     body_text = driver.find_element(By.TAG_NAME, "body").text
#     all_text_data.append({
#         "title": driver.title,
#         "url": url,
#         "text": body_text
#     })

#      # 테이블 데이터 수집
#     # 테이블 데이터 수집
#     try:
#         # 테이블이 존재할 때까지 최대 10초 대기
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.XPATH, "//table"))
#         )
#         tables = driver.find_elements(By.TAG_NAME, "table")  # 모든 table 태그 찾기
#         for table_index, table in enumerate(tables):
#             rows = table.find_elements(By.XPATH, ".//tr")  # 각 table의 모든 tr 태그 찾기
#             table_data = []

#             for row_index, row in enumerate(rows):
#                 row_data = []
#                 cells = row.find_elements(By.XPATH, ".//td | .//th")  # td와 th 태그 찾기
#                 for cell in cells:
#                     try:
#                         # 셀 내부 모든 텍스트를 가져오기 (중첩된 태그 포함)
#                         cell_text = cell.get_attribute("innerText").strip()
#                         row_data.append(cell_text if cell_text else "N/A")
#                     except StaleElementReferenceException:
#                         print("StaleElementReferenceException 발생. 셀을 건너뜁니다.")
#                         row_data.append("N/A")
#                 table_data.append(row_data)

#             all_table_data.append({
#                 "title": driver.title,
#                 "url": url,
#                 "table_index": table_index,
#                 "table_data": table_data
#             })

#     except TimeoutException:
#         # 테이블이 없는 경우 예외 처리
#         print(f"페이지에 테이블이 없습니다: {url}")
#         all_table_data.append({
#             "title": driver.title,
#             "url": url,
#             "table_data": "테이블 없음"
#         })
        
#     # elements = driver.find_elements(By.XPATH, "//*")  # 모든 태그를 찾기 위한 XPath

#     # # 태그 데이터 수집
#     # page_tag_data = {
#     #     "title": driver.title,
#     #     "url": url,
#     #     "tags": []
#     # }
#     # elements = driver.find_elements(By.XPATH, "//*")
#     # for element in elements:
#     #     try:
#     #         tag_info = {
#     #             "tag_name": element.tag_name,
#     #             "attributes": element.get_attribute("outerHTML"),
#     #             "text": element.text.strip()
#     #         }
#     #         page_tag_data["tags"].append(tag_info)
#     #     except StaleElementReferenceException:
#     #         print("StaleElementReferenceException 발생. 태그를 건너뜁니다.")
#     #         continue

#     # all_tag_data.append(page_tag_data)

#     time.sleep(0.5)
#     if i==4:
#         break

# print('전체 데이터',all_text_data)
# print('table 행 별 데이터',all_table_data)

# # 테이블 데이터를 JSON 파일로 저장
# table_file_name=f'table_data_{time.localtime().tm_mon}{time.localtime().tm_mday}{time.localtime().tm_hour}{time.localtime().tm_min}{time.localtime().tm_sec}'
# with open(f"{table_file_name}.json", "w", encoding="utf-8") as f:
#     json.dump(all_table_data, f, ensure_ascii=False, indent=4)
# print(f"JSON 파일 저장 완료: {table_file_name}.json")
# 동작 확인을 위해 잠시 대기 후 종료
time.sleep(7)

#driver.quit()


Page title: 직무별 채용 정보 - 내가 원하는 직군의 취업 공고 찾기 | 잡코리아
현재 페이지: 1
추출된 href (1/40): https://www.jobkorea.co.kr/Recruit/GI_Read/46104266?rPageCode=SL&logpath=21&sn=6
추출된 href (2/40): https://www.jobkorea.co.kr/Recruit/GI_Read/46099393?rPageCode=SL&logpath=21&sn=6
추출된 href (3/40): https://www.jobkorea.co.kr/Recruit/GI_Read/46092442?rPageCode=SL&logpath=21&sn=6
추출된 href (4/40): https://www.jobkorea.co.kr/Recruit/GI_Read/46088416?rPageCode=SL&logpath=21&sn=6
추출된 href (5/40): https://www.jobkorea.co.kr/Recruit/GI_Read/46085654?rPageCode=SL&logpath=21&sn=6
추출된 href (6/40): https://www.jobkorea.co.kr/Recruit/GI_Read/46076951?rPageCode=SL&logpath=21&sn=6
추출된 href (7/40): https://www.jobkorea.co.kr/Recruit/GI_Read/46058290?rPageCode=SL&logpath=21&sn=6
추출된 href (8/40): https://www.jobkorea.co.kr/Recruit/GI_Read/46051944?rPageCode=SL&logpath=21&sn=6
추출된 href (9/40): https://www.jobkorea.co.kr/Recruit/GI_Read/46052361?rPageCode=SL&logpath=21&sn=6
추출된 href (10/40): https://www.jobkorea.co.kr/Recruit/GI_R